None, True and False are all singleton objects

In [1]:
NoneType = type(None)

In [2]:
n1 = NoneType()
n2 = NoneType()

In [3]:
n1, n2, id(n1), id(n2)

(None, None, 140723480452320, 140723480452320)

In [4]:
id(bool([])), id(bool("")), id(bool(False)), 

(140723480406384, 140723480406384, 140723480406384)

we can create singleton without metaclasses

first we will override creating the instance

In [5]:
class Hundred:
    def __new__(cls):
        new_instance = super().__new__(cls)
        setattr(new_instance, "name", "hundred")
        setattr(new_instance, "value", 100)
        return new_instance

In [6]:
h1 = Hundred()
h2 = Hundred()

In [8]:
vars(h1), h1 is h2

({'name': 'hundred', 'value': 100}, False)

let's make it into a singleton. we will use the class itself as a storage mechanism for the single instance. We will store instance as a class variable.

In [10]:
class Hundred:
    _existing_instance = None
    
    def __new__(cls):
        if not cls._existing_instance:
            print("creating new instance")
            new_instance = super().__new__(cls)
            setattr(new_instance, "name", "hundred")
            setattr(new_instance, "value", 100)
            cls._existing_instance = new_instance
        else:
            print("instance exists already...")
        return cls._existing_instance

In [11]:
h1 = Hundred()
h2 = Hundred()

creating new instance
instance exists already...


In [12]:
h1 is h2

True

if we want to have many classes as singleton, we will use a metaclass

let's just override the call method first

In [13]:
class Singleton(type):
    def __call__(cls, *args, **kwargs):
        print(f"Request received to create an instance of class: {cls}..")
        return super().__call__(*args, **kwargs)

In [14]:
class Hundred(metaclass=Singleton):
    value = 100

In [16]:
h = Hundred()

Request received to create an instance of class: <class '__main__.Hundred'>..


In [17]:
h.value

100

let's make it singleton

class gets passed into the call method. we can use that to store the instance directly on the class

In [24]:
class Singleton(type):
    def __call__(cls, *args, **kwargs):
        print(f"Request received to create an instance of class: {cls}...")
        if getattr(cls, "_existing_instance", None) is None:
            print("Creating instance for the first time...")
            setattr(cls, "_existing_instance", super().__call__(*args, **kwargs))
        else:
            print("using existing instance...")
        return cls._existing_instance

In [25]:
class Hundred(metaclass=Singleton):
    value = 100

In [26]:
h1 = Hundred()

Request received to create an instance of class: <class '__main__.Hundred'>...
Creating instance for the first time...


In [27]:
h2 = Hundred()

Request received to create an instance of class: <class '__main__.Hundred'>...
using existing instance...


In [28]:
h1 is h2

True

In [29]:
vars(Hundred)

mappingproxy({'__module__': '__main__',
              'value': 100,
              '__dict__': <attribute '__dict__' of 'Hundred' objects>,
              '__weakref__': <attribute '__weakref__' of 'Hundred' objects>,
              '__doc__': None,
              '_existing_instance': <__main__.Hundred at 0x1e531c58dc8>})

In [30]:
h1.value, h2.value

(100, 100)

In [31]:
class Thousand(metaclass=Singleton):
    value = 1000

In [32]:
t1 = Thousand()
t2 = Thousand()

Request received to create an instance of class: <class '__main__.Thousand'>...
Creating instance for the first time...
Request received to create an instance of class: <class '__main__.Thousand'>...
using existing instance...


In [33]:
t1 is t2

True

In [34]:
t1.value, t2.value

(1000, 1000)

In [35]:
t1 is h1

False

let's try it with inheritance

In [39]:
class HundredFold(Hundred):
    value = 100 * 100

In [40]:
type(HundredFold)

__main__.Singleton

In [41]:
vars(HundredFold)

mappingproxy({'__module__': '__main__', 'value': 10000, '__doc__': None})

In [42]:
vars(Hundred) # we have an exsisting instance in the parent class already

mappingproxy({'__module__': '__main__',
              'value': 100,
              '__dict__': <attribute '__dict__' of 'Hundred' objects>,
              '__weakref__': <attribute '__weakref__' of 'Hundred' objects>,
              '__doc__': None,
              '_existing_instance': <__main__.Hundred at 0x1e531c58dc8>})

In [43]:
hf1 = HundredFold()

Request received to create an instance of class: <class '__main__.HundredFold'>...
using existing instance...


In [44]:
hf1.value

100

let's fix it

In [45]:
class Singleton(type):
    instances = {}
    def __call__(cls, *args, **kwargs):
        print(f"Request received to create an instance of class: {cls}...")
        _existing_instance = Singleton.instances.get(cls, None)
        if _existing_instance is None:
            print("Creating instance for the first time...")
            Singleton.instances[cls] = super().__call__(*args, **kwargs)
        else:
            print("using existing instance...")
        return Singleton.instances.get(cls)

In [46]:
class Hundred(metaclass=Singleton):
    value = 100

class Thousand(metaclass=Singleton):
    value = 1000
    
class HundredFold(Hundred):
    value = 100 * 100

In [47]:
h1 = Hundred()
h2 = Hundred()

Request received to create an instance of class: <class '__main__.Hundred'>...
Creating instance for the first time...
Request received to create an instance of class: <class '__main__.Hundred'>...
using existing instance...


In [48]:
h1 is h2

True

In [49]:
t1 = Thousand()
t2 = Thousand()

Request received to create an instance of class: <class '__main__.Thousand'>...
Creating instance for the first time...
Request received to create an instance of class: <class '__main__.Thousand'>...
using existing instance...


In [50]:
t1 is t2, t1 is h1

(True, False)

In [51]:
hf1 = HundredFold()
hf2 = HundredFold()

Request received to create an instance of class: <class '__main__.HundredFold'>...
Creating instance for the first time...
Request received to create an instance of class: <class '__main__.HundredFold'>...
using existing instance...


In [52]:
hf1 is hf2, hf1 is h1

(True, False)

In [53]:
h1.value, hf1.value

(100, 10000)